In [37]:
import sys
sys.path.insert(1, '../../../scripts/')
from s3_support import *

import pandas as pd

In [38]:
cols = ['org', 'date', 'calls_to_action', 'iframe_source_blackbaud',
         'iframe_source_classy', 'iframe_source_donordrive', 'iframe_source_engagingnetworks',
         'iframe_source_facebook', 'iframe_source_giveeffect','iframe_source_givelively',
         'iframe_source_instagram','iframe_source_mobilecause','iframe_source_networkforgood',
         'iframe_source_onecause','iframe_source_paypal','iframe_source_qgiv',
         'iframe_source_stripe','iframe_source_twitter','image_count',
         'internal_links','link_target_blackbaud','link_target_classy',
         'link_target_donordrive','link_target_engagingnetworks','link_target_facebook',
         'link_target_giveeffect','link_target_givelively','link_target_instagram',
         'link_target_mobilecause','link_target_networkforgood','link_target_onecause',
         'link_target_paypal','link_target_qgiv','link_target_stripe',
         'link_target_twitter','outbound_links','script_source_blackbaud',
         'script_source_classy','script_source_donordrive','script_source_engagingnetworks',
         'script_source_facebook','script_source_giveeffect','script_source_givelively',
         'script_source_instagram','script_source_mobilecause','script_source_networkforgood',
         'script_source_onecause','script_source_paypal','script_source_qgiv',
         'script_source_stripe','script_source_twitter','url','word_count',
         'iframe_source_securegive', 'link_target_securegive', 'script_source_securegive']

In [46]:
df = get_dataframe_from_file("qgiv-stats-data", "org_website.csv")

In [47]:
len(df), df.groupby('date')['org'].count()

(90162,
 date
 2024-12-15    35857
 2024-12-16    54305
 Name: org, dtype: int64)

In [48]:
#df = get_dataframe_from_file("qgiv-stats-data", "org_website.csv")
#df['date'] = pd.to_datetime(df['date'])

df['date'] = '2024-12-15'
df['date'] = pd.to_datetime(df['date'])

df = df[(df['url'].str.startswith('http'))&(~df['url'].str.startswith('https://twitter.com'))&(~df['url'].str.startswith('https://www.facebook.com'))&(~df['url'].str.startswith('https://www.linkedin.com'))&(~df['url'].str.startswith('https://www.google.com'))]

In [49]:
print(len(df))
print(len(df.drop_duplicates('url')))

df.drop_duplicates('url', inplace=True)

89397
88916


In [50]:
df = df[~df['org'].isna()]
len(df)

88916

In [51]:
for c in cols:
    if c != 'date' and c != 'url':
        if c not in df.columns:
            df[c] = 0
        else:
            df[c] = df[c].fillna(0).astype('int')
    elif c == 'url':
        df[c] = df[c].apply(lambda x: x.replace(',', '').replace('\n', '')[:200])

In [52]:
df.tail(3)

,url,outbound_links,internal_links,calls_to_action,word_count,image_count,script_source_qgiv,script_source_paypal,script_source_blackbaud,script_source_classy,...,iframe_source_giveeffect,iframe_source_engagingnetworks,iframe_source_donordrive,iframe_source_stripe,iframe_source_securegive,iframe_source_facebook,iframe_source_twitter,iframe_source_instagram,org,date
90159,http://www.liftyouthcenter.org/foreveryoung,39,11,3,3355,23,0,0,0,0,...,0,0,0,0,0,0,0,0,456360,2024-12-15
90160,https://www.theparc.net/,17,24,8,3391,13,0,0,0,0,...,0,0,0,0,0,0,0,0,456374,2024-12-15
90161,https://www.theparc.net/subscribe,8,23,7,844,2,0,0,0,0,...,0,0,0,0,0,0,0,0,456374,2024-12-15


In [53]:
save_dataframe_to_file("qgiv-stats-data", "org_website.csv", df[cols])

uploading to S3
Done


In [54]:
redshift_query_write('''copy org_websites
                        from 's3://qgiv-stats-data/org_website.csv'
                        iam_role 'arn:aws:iam::637885584661:role/AWSRoleForRedshift'
                        emptyasnull
                        blanksasnull
                        fillrecord
                        delimiter ','
                        ignoreheader 1
                        region 'us-east-1'
                        ''', schema='production')

In [55]:
# verify
redshift_query_read("select date, count(date) as count from org_websites group by date order by date desc", schema='production').head()

,date,count
0,2024-12-15,88916
1,2024-12-01,87637
2,2024-11-15,85264
3,2024-11-01,83903
4,2024-10-15,82211


In [12]:
sources = [c.replace('iframe_source_', '') for c in df.columns if 'iframe' in c]
sources

['blackbaud',
 'classy',
 'donordrive',
 'engagingnetworks',
 'facebook',
 'giveeffect',
 'givelively',
 'instagram',
 'mobilecause',
 'networkforgood',
 'onecause',
 'paypal',
 'qgiv',
 'securegive',
 'stripe',
 'twitter']

In [13]:
source_presence_data = []
for source in sources:
    rel_cols = [c for c in df.columns if source in c]
    presence = df[rel_cols].sum().sum() / float(len(df))
    source_presence_data.append({
        'source': source,
        'presence': presence * 100.
    })
pd.DataFrame(source_presence_data)[['source', 'presence']]

,source,presence
0,blackbaud,0.998263
1,classy,0.668966
2,donordrive,0.324111
3,engagingnetworks,0.000000
4,facebook,238.774081
5,giveeffect,0.000000
6,givelively,0.145202
7,instagram,102.872922
8,mobilecause,0.077787
9,networkforgood,2.828843


In [20]:
rel_cols = [c for c in df.columns if 'donordrive' in c]
df[rel_cols].sum().sum() / len(df)

0.000513319284595018

In [21]:
gen_stats_cols = ['calls_to_action', 'internal_links', 'outbound_links', 'word_count', 'image_count']

df[gen_stats_cols].mean().reset_index()

,index,0
0,calls_to_action,4.318987
1,internal_links,73.333955
2,outbound_links,16.733128
3,word_count,85701.730102
4,image_count,12.677311
